## validation split 방식과 k-fold 방식에 차이

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [2]:

# 데이터 로드 및 전처리
# Pandas DataFrame에서는 iloc를 사용해야 위치 기반 슬라이싱이 가능합니다.
data = pd.read_csv('https://raw.githubusercontent.com/pykwon/python/refs/heads/master/testdata_utf8/pima-indians-diabetes.data.csv',
                header=None, dtype=np.float32) # header=None 추가
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

print('x 데이터 샘플:\n', x[:3])
print('\ny 데이터 샘플:\n', y[:3])

# 모델 구성 함수 (함수명을 build_model로 통일)
def build_model():
    model = Sequential([
        Input(shape=(8,)),
        Dense(units=64, activation='relu'),
        Dense(units=32, activation='relu'),
        Dense(units=1, activation='sigmoid'),
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

x 데이터 샘플:
 [[  6.    148.     72.     35.      0.     33.6     0.627  50.   ]
 [  1.     85.     66.     29.      0.     26.6     0.351  31.   ]
 [  8.    183.     64.      0.      0.     23.3     0.672  32.   ]]

y 데이터 샘플:
 [1. 0. 1.]


## validation_split 방식 사용

In [4]:
from tabnanny import verbose
from libmambapy import History


model_val = build_model()
history_val = model_val.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
val_acc = history_val.history['val_accuracy'][-1]

## KFold 방식 사용

In [5]:
from param import batch_watch


kf = KFold(n_splits=5, shuffle=True, random_state=42)
kfold_accuracies = []

for train_idx, val_idx in kf.split(x):
    # 학습용, 검증용 데이터 분할
    x_train, x_val = x[train_idx], x[val_idx]
    # y_val 변수명 오타 수정 (x_val -> y_val)
    y_train, y_val = y[train_idx], y[val_idx]

    # 모델 생성 및 학습
    model_kf = build_model()
    model_kf.fit(x_train, y_train, epochs=50, batch_size=32, verbose=0)

    # 예측 및 정확도 계산
    y_pred = model_kf.predict(x_val)
    y_pred_label = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_val, y_pred_label)
    kfold_accuracies.append(acc)
    

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002470C3702C0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


## 비교 출력

In [6]:
print(f'[validation_split] 마지막 검증 정확도: {val_acc:.4f}')
print(f'[KFold] 각 fold의 정확도: {np.round(kfold_accuracies, 4)}')
print(f'[KFold] 평균 정확도: {np.mean(kfold_accuracies):.4f}')

[validation_split] 마지막 검증 정확도: 0.6688
[KFold] 각 fold의 정확도: [0.6883 0.7532 0.6753 0.6732 0.6601]
[KFold] 평균 정확도: 0.6900
